In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
## 1. create an imbalanted binary classification dataset
X,y = make_classification(n_samples=1000, n_features=10, n_informative=2,
                          n_redundant=8, weights=[0.9,0.1],
                          flip_y=0, random_state=42)
np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
## Split the datasets into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Experiment 1: train Logistic-Regression-Classifier**

In [5]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train,y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test,y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       270
           1       0.78      0.60      0.68        30

    accuracy                           0.94       300
   macro avg       0.87      0.79      0.82       300
weighted avg       0.94      0.94      0.94       300



**Experiment 2: Train Random-Forest-Classifier**

In [6]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train,y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       1.00      0.63      0.78        30

    accuracy                           0.96       300
   macro avg       0.98      0.82      0.88       300
weighted avg       0.96      0.96      0.96       300



**Experiment 3: Train XGBoost**

In [7]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train,y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.91      0.70      0.79        30

    accuracy                           0.96       300
   macro avg       0.94      0.85      0.89       300
weighted avg       0.96      0.96      0.96       300



**Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost**

In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train,y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([614, 614], dtype=int64))

In [9]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       270
           1       0.76      0.83      0.79        30

    accuracy                           0.96       300
   macro avg       0.87      0.90      0.88       300
weighted avg       0.96      0.96      0.96       300



## Track Experiments Using MLflow

In [10]:
models = [
    (
        'LogisticRegression',
        LogisticRegression(C=1, solver='liblinear'),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        'Random Forest',
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        'XGBClassifier',
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        'XGBClassifier With SMOTE',
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train_res,y_train_res),
        (X_test,y_test)
    )
]

In [11]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [12]:
reports

[{'0': {'precision': 0.9566787003610109,
   'recall': 0.9814814814814815,
   'f1-score': 0.9689213893967094,
   'support': 270.0},
  '1': {'precision': 0.782608695652174,
   'recall': 0.6,
   'f1-score': 0.6792452830188679,
   'support': 30.0},
  'accuracy': 0.9433333333333334,
  'macro avg': {'precision': 0.8696436980065925,
   'recall': 0.7907407407407407,
   'f1-score': 0.8240833362077886,
   'support': 300.0},
  'weighted avg': {'precision': 0.939271699890127,
   'recall': 0.9433333333333334,
   'f1-score': 0.9399537787589253,
   'support': 300.0}},
 {'0': {'precision': 0.956989247311828,
   'recall': 0.9888888888888889,
   'f1-score': 0.9726775956284153,
   'support': 270.0},
  '1': {'precision': 0.8571428571428571,
   'recall': 0.6,
   'f1-score': 0.7058823529411765,
   'support': 30.0},
  'accuracy': 0.95,
  'macro avg': {'precision': 0.9070660522273426,
   'recall': 0.7944444444444445,
   'f1-score': 0.8392799742847958,
   'support': 300.0},
  'weighted avg': {'precision': 0.94

In [13]:
import mlflow

In [24]:
mlflow.set_experiment('Anomaly Detection')
mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')

for i, element in enumerate(models):
    model_name = element[0] # first element is a model-name->in models
    model = element[1] # second element in models is model
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param('model_name', model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])
        
        if 'XGB' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')
        

2024/11/05 19:46:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/05 19:46:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/950187950321061705/runs/44e6c1af1d4e4a44bdf30281c072248d.
2024/11/05 19:46:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/950187950321061705.
2024/11/05 19:46:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/05 19:46:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/950187950321061705/runs/4fde72c3d5324312a09a104e2db6411c.
2024/11/05 19:46:11 INFO mlflow.tracking._tracking_service.client: 🧪 View expe

**Register the Model**

In [ ]:
run_id = input('Enter Run ID:') # we take the model Run-ID
model_uri = f'runs:/{run_id}/model'

result = mlflow.register_model(
    model_uri, model_name
)

Successfully registered model 'XGB-Smote'.
Created version '1' of model 'XGB-Smote'.


**Load the Model**

In [18]:
model_name = 'XGB-Smote'
model_version = 1
model_uri = f'models:/{model_name}@challenger'

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 1, 0, 0])

**Now it's time for transation from dev-env -> production**

In [19]:
dev_model_uri = f'models:/{model_name}@challenger'
prod_model = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1730885523025, current_stage='None', description=None, last_updated_timestamp=1730885523025, name='anomaly-detection-prod', run_id='185e8a47d8ad4a1fb7f2dea26a733d27', run_link=None, source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [21]:
model_uri = f'models:/{prod_model}@champion'

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 1, 0, 0])